# Imports

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
from google.colab import files
from tqdm.notebook import tqdm

#Pipeline de procesado

## Subir el csv

In [8]:
uploaded = files.upload()
for filename in uploaded.keys():
  dataframe = pd.read_csv(filename)
  dataframe.head()

Saving english_df_part_2.csv to english_df_part_2.csv


In [9]:
dataframe.head()

,id,review,type_review,date,language,views,votes,score
0,dertyp,"You know a game is good, when you keep thinkin...",expanded,2020-07-12,English,4,3,10
1,AgentTapir,This game really touched me deeply. I had to t...,expanded,2020-07-12,English,4,3,10
2,Aluminium,"Just finished it after 37h. Technically, best ...",normal,2020-07-12,English,4,3,10
3,Normanfletcher,The best game I've ever played. One of the bes...,expanded,2020-07-12,English,4,3,10
4,jank_king20,Don’t believe the babies crying about this gam...,normal,2020-07-12,English,4,3,10


## Calcular sentiment análisis

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

MODEL = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

reviews = dataframe["review"]
for index, text in  tqdm(enumerate(reviews), total=len(reviews)):
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    output = model(**encoded_input)
    scores = softmax(output.logits.detach().numpy(), axis=1)[0]
    labels = config.id2label

    for label, score in zip(labels, scores):
        dataframe.at[index, label] = score

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

  0%|          | 0/2421 [00:00<?, ?it/s]

In [11]:
dataframe.head()

,id,review,type_review,date,language,views,votes,score,0,1
0,dertyp,"You know a game is good, when you keep thinkin...",expanded,2020-07-12,English,4,3,10,0.001117,0.998882
1,AgentTapir,This game really touched me deeply. I had to t...,expanded,2020-07-12,English,4,3,10,0.001103,0.998897
2,Aluminium,"Just finished it after 37h. Technically, best ...",normal,2020-07-12,English,4,3,10,0.002018,0.997982
3,Normanfletcher,The best game I've ever played. One of the bes...,expanded,2020-07-12,English,4,3,10,0.001125,0.998875
4,jank_king20,Don’t believe the babies crying about this gam...,normal,2020-07-12,English,4,3,10,0.001073,0.998927


In [12]:
dataframe = dataframe.rename(columns={0: "negative", 1: "positive"})
dataframe.head(15)


,id,review,type_review,date,language,views,votes,score,negative,positive
0,dertyp,"You know a game is good, when you keep thinkin...",expanded,2020-07-12,English,4,3,10,0.001117,0.998882
1,AgentTapir,This game really touched me deeply. I had to t...,expanded,2020-07-12,English,4,3,10,0.001103,0.998897
2,Aluminium,"Just finished it after 37h. Technically, best ...",normal,2020-07-12,English,4,3,10,0.002018,0.997982
3,Normanfletcher,The best game I've ever played. One of the bes...,expanded,2020-07-12,English,4,3,10,0.001125,0.998875
4,jank_king20,Don’t believe the babies crying about this gam...,normal,2020-07-12,English,4,3,10,0.001073,0.998927
5,PatrykPlayer,GREAT ! They create a Masterpiece. After 7 yea...,expanded,2020-07-12,English,4,3,10,0.001092,0.998908
6,XeniaRom,Genius masterpiece. I enjoyed every second of ...,expanded,2020-07-12,English,4,3,10,0.001074,0.998926
7,Assassins1308,"I really like this game, starting from the gam...",expanded,2020-07-12,English,4,3,10,0.001065,0.998935
8,N-Shifter,"Absolute masterpiece, 30+hours on my first pla...",expanded,2020-07-12,English,4,3,10,0.001200,0.998800
9,dramon,Much like the harrowing journey and violence s...,normal,2020-07-12,English,4,3,10,0.001093,0.998907


## Descargar los resultados de salida

In [13]:
filename = f'sent_english_part_.csv'
dataframe.to_csv(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>